In [ ]:
using Distributions
using Optim

In [ ]:
function generate_data(N)
    delta1, delta2 = -3 , -2
    beta1, beta2 = [1 4], [2 3]
    rho = -2
    lambda = exp(rho)/(1+exp(rho))

    Y = zeros(N,3)
    X = zeros(N,3)

    for i=1:N
        x1,x2 = rand(Normal(0.0, 1.0),3)
        e1,e2 = rand(Logistic(),2)
        select = rand(Categorical([lambda, (1-lambda)]))

        #see logit response i.e. definitely 11
        eq = []
        for y_init=[[0 0],[1 1],[0 1],[1 0]]
            y1,y2 = sum(beta1.*[1 x1]) + delta1*y_init[2] + e1 >= 0, sum(beta2.*[1 x2]) +delta2*y_init[1] + e2 >= 0

            if false
                println( "$(1*y1) $(1*y2),$(y_init)" )
                println(sum(beta1.*[1 x1]) + delta1*y_init[2] + e1," ", sum(beta2.*[1 x2]) +delta2*y_init[1] + e2)
                println("-----")
            end

            if (y1==y_init[1]) & (y2==y_init[2])
                append!(eq,[y_init])
            end
        end
        
        y1, y2 = eq[1]
        if size(eq)[1] >= 2
            y1, y2 = eq[select]
        end
        Y[i,:] =[y1 y2]
        X[i,:] =[x1 x2]
    end
    return Y,X
end
